# L5: Agentic RAG & External Memory


## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

Letta agents persist information over time and restarts by saving data to a database. These lessons do not require past information. To enable a clean restart, the database is cleared before starting the lesson.

In [ ]:
!rm  -f ~/.letta/sqlite.db

## Section 0: Setup a client 

In [ ]:
from helper import nb_print, load_env
from dotenv import load_dotenv

_ = load_dotenv()

In [ ]:
from letta import create_client 

client = create_client()

In [ ]:
from letta.schemas.llm_config import LLMConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini"))

## Section 1: Loading data into archival memory 

In [ ]:
source = client.create_source("employee_handbook")
source

In [ ]:
client.load_file_into_source(
    filename="handbook.pdf", 
    source_id=source.id
)

In [ ]:
agent_state = client.create_agent()

In [ ]:
client.attach_source_to_agent(
    agent_id=agent_state.id, 
    source_id=source.id
)

In [ ]:
client.list_attached_sources(agent_state.id)

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Search archival for our company's vacation policies", 
    role = "user"
) 
nb_print(response.messages)

## Section 2: Connecting data via tools

In [ ]:
def query_birthday_db(self, name: str): 
    """
    This tool queries an external database to 
    lookup the birthday of someone given their name.

    Args: 
        name (str): The name to look up 

    Returns: 
        birthday (str): The birthday in mm-dd-yyyy format
    
    """
    my_fake_data = {
        "bob": "03-06-1997", 
        "sarah": "03-06-1997"
    } 
    name = name.lower() 
    if name not in my_fake_data: 
        return None
    else: 
        return my_fake_data[name]

In [ ]:
birthday_tool = client.create_tool(query_birthday_db)

In [ ]:
birthday_tool

In [ ]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name="birthday_agent", 
    tools=[birthday_tool.name], 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona="You are a agent with access to a birthday_db " \
        + "that you use to lookup information about users' birthdays."
    )
)

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "When is my birthday?", 
    role = "user"
) 
nb_print(response.messages)

### Loading tools from Langchain

In [ ]:
import getpass
import os
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [ ]:
from langchain_community.tools import TavilySearchResults
from letta.schemas.tool import Tool

search = TavilySearchResults()

In [ ]:
search.run("What's Obama's first name?") 

In [ ]:
# convert the tool to Letta Tool 
search_tool = Tool.from_langchain(TavilySearchResults())

In [ ]:
# persist the tool 
client.add_tool(search_tool)

In [ ]:
research_agent_persona = f"""
You have access to a web via a {search_tool.name} tool. 
Use this tool to respond to users' questions, by summarizing the
{search_tool.name} 
and also providing the `url` that the information was from as a 
reference. 

<Example> 
User: 'What is Obama's first name?' 
Assistant: 'Obama's first name is Barack.

Sources:
[1] https://www.britannica.com/biography/Barack-Obama
[2] https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States'
</Example>
Your MUST provide URLs that you used to generate the answer, or you will be terminated. 

"""

In [ ]:
agent_state = client.create_agent(
    name="research_agent", 
    tools=[search_tool.name], 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona=research_agent_persona
    )
)

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Who founded OpenAI? ", 
    role = "user"
) 
nb_print(response.messages)

In [ ]:
from letta.schemas.llm_config import LLMConfig

agent_state = client.create_agent(
    name="gpt4_search_agent", 
    tools=[search_tool.name], 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona=research_agent_persona
    ),
    #llm_config=LLMConfig.default_config('gpt-4')  # uncomment if you are not getting desired results. Note that this is much more expensive than gpt-4o-mini
    llm_config=LLMConfig.default_config('gpt-4o-mini')
)

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Who founded OpenAI? ", 
    role = "user"
) 
nb_print(response.messages)